# DATA CLEANING (2 of 3 Documents)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import datetime
import sklearn as sk
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from pandas.plotting import scatter_matrix

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('matplotlib version:', mpl.__version__)

%matplotlib inline

numpy version: 1.14.3
pandas version: 0.23.0
scikit-learn version: 0.19.1
matplotlib version: 2.2.2


In [24]:
df_orders = pd.read_csv("../Datasets/olist_orders_dataset.csv")
df_customers = pd.read_csv("../Datasets/olist_customers_dataset.csv")
df_geoloc = pd.read_csv("../Datasets/olist_geolocation_dataset.csv")
df_items = pd.read_csv("../Datasets/olist_order_items_dataset.csv")
df_payments = pd.read_csv("../Datasets/olist_order_payments_dataset.csv")
df_seller = pd.read_csv("../Datasets/olist_sellers_dataset.csv")
df_product = pd.read_csv("../Datasets/olist_products_dataset.csv")
df_product_translation = pd.read_csv("../Datasets/product_category_name_translation.csv")

In [25]:
df_orders_copy = df_orders.copy()
df_orders_copy["order_purchase_timestamp"] = pd.to_datetime(df_orders["order_purchase_timestamp"], format='%Y-%m-%d %H:%M:%S')
df_orders_copy["order_approved_at"] = pd.to_datetime(df_orders["order_approved_at"], format='%Y-%m-%d %H:%M:%S')
df_orders_copy["order_delivered_carrier_date"] = pd.to_datetime(df_orders["order_delivered_carrier_date"], format='%Y-%m-%d %H:%M:%S')
df_orders_copy["order_delivered_customer_date"] = pd.to_datetime(df_orders["order_delivered_customer_date"], format='%Y-%m-%d %H:%M:%S')
df_orders_copy["order_estimated_delivery_date"] = pd.to_datetime(df_orders["order_estimated_delivery_date"], format='%Y-%m-%d %H:%M:%S')

In [26]:
df_orders_copy['day_name']=df_orders_copy['order_purchase_timestamp'].dt.weekday_name
df_orders_copy['year']=df_orders_copy['order_purchase_timestamp'].dt.year
df_orders_copy['day']=df_orders_copy['order_purchase_timestamp'].dt.day
df_orders_copy['month']=df_orders_copy['order_purchase_timestamp'].dt.month
df_orders_copy['hour']=df_orders_copy['order_purchase_timestamp'].dt.hour

In [27]:
df_orders_copy['day_categories'] = df_orders_copy['day_name'].map({'Monday':0, 'Tuesday':1, 'Wednesday':2, 'Thursday':3, 'Friday':4, 'Saturday':5, 'Sunday':6})

In [28]:
df_orders_copy = df_orders_copy.drop('day_name', axis=1)

In [30]:
df_orders_copy = df_orders_copy.drop('order_approved_at', axis=1)
df_orders_copy = df_orders_copy.drop('order_delivered_carrier_date', axis=1)
df_orders_copy = df_orders_copy.drop('order_delivered_customer_date', axis=1)
df_orders_copy = df_orders_copy.drop('order_estimated_delivery_date', axis=1)

In [31]:
df_items['total_order_value'] = ((df_items['order_item_id']*df_items['price']) + df_items['order_item_id']*df_items['freight_value'])
df_join_item_order = df_orders_copy.join(df_items.set_index('order_id'), on='order_id')

In [32]:
        df_join_item_order = df_join_item_order.join(df_product.set_index('product_id'), on='product_id')

In [21]:
df_join_item_order.count()

order_purchase_timestamp         112650
year                             112650
day                              112650
month                            112650
hour                             112650
day_categories                   112650
total_order_value                112650
product_category_name_english    112650
customer_state                   112650
state_number                     112650
product_category                 112650
dtype: int64

In [33]:
df_join_item_order = df_join_item_order.drop('product_name_lenght', axis=1)
df_join_item_order = df_join_item_order.drop('product_length_cm', axis=1)
df_join_item_order = df_join_item_order.drop('product_height_cm', axis=1)
df_join_item_order = df_join_item_order.drop('product_width_cm', axis=1)
df_join_item_order = df_join_item_order.drop('product_photos_qty', axis=1)
df_join_item_order = df_join_item_order.drop('product_weight_g', axis=1)
df_join_item_order = df_join_item_order.drop('product_description_lenght', axis=1)
df_join_item_order = df_join_item_order.drop(['order_item_id', 'price', 'freight_value', 'seller_id'], axis=1)

In [34]:
df_join_item_order = df_join_item_order.join(df_product_translation.set_index('product_category_name'), on='product_category_name')

In [35]:
df_join_item_order = df_join_item_order[df_join_item_order['total_order_value'].isnull() == False]
df_join_item_order['product_category_name_english'] = df_join_item_order['product_category_name_english'].fillna("Unidentified")
df_join_item_order = df_join_item_order.join(df_customers.set_index('customer_id'), on='customer_id')

Deleting the null value (above) and filling the missing value

In [36]:
df_join_item_order = df_join_item_order.drop(['product_category_name', 'product_id'], axis=1)
df_join_item_order = df_join_item_order.drop(['customer_unique_id', 'customer_city', 'customer_zip_code_prefix'], axis=1)
df_join_item_order = df_join_item_order.drop(['customer_id', 'order_status'], axis=1)

In [37]:
df_join_item_order['state_number'] = df_join_item_order['customer_state'].map({'SP': 0, 'RJ': 1, 'MG': 2, 'RS': 3, 'PR': 4, 'SC': 5, 'BA': 6, 'DF': 7, 'GO': 8, 'ES': 9, 'PE': 10, 'CE': 11, 'PA': 12, 'MT': 13, 'MA': 14, 'MS': 15, 'PB': 16, 'PI': 17, 'RN': 18, 'AL': 19, 'SE': 20, 'TO': 21, 'RO': 22, 'AM': 23, 'AC': 24, 'AP': 25, 'RR': 26})
df_join_item_order['product_category'] = df_join_item_order['product_category_name_english'].map({'health_beauty': 0, 'computers_accessories': 1, 'auto': 2, 'bed_bath_table': 3, 'furniture_decor': 4, 'sports_leisure': 5, 'perfumery': 6, 'housewares': 7, 'telephony': 8, 'watches_gifts': 9, 'food_drink': 10, 'baby': 11, 'stationery': 12, 'tablets_printing_image': 13, 'toys': 14, 'fixed_telephony': 15, 'garden_tools': 16, 'fashion_bags_accessories': 17, 'small_appliances': 18, 'consoles_games': 19, 'audio': 20, 'fashion_shoes': 21, 'cool_stuff': 22, 'luggage_accessories': 23, 'air_conditioning': 24, 'construction_tools_construction': 25, 'kitchen_dining_laundry_garden_furniture': 26, 'costruction_tools_garden': 27, 'fashion_male_clothing': 28, 'pet_shop': 29, 'office_furniture': 30, 'market_place': 31, 'electronics': 32, 'home_appliances': 33, 'party_supplies': 34, 'home_confort': 35, 'costruction_tools_tools': 36, 'agro_industry_and_commerce': 37, 'furniture_mattress_and_upholstery': 38, 'books_technical': 39, 'home_construction': 40, 'musical_instruments': 41, 'furniture_living_room': 42, 'construction_tools_lights': 43, 'industry_commerce_and_business': 44, 'food': 45, 'art': 46, 'furniture_bedroom': 47, 'books_general_interest': 48, 'construction_tools_safety': 49, 'fashion_underwear_beach': 50, 'fashion_sport': 51, 'signaling_and_security': 52, 'computers': 53, 'christmas_supplies': 54, 'fashio_female_clothing': 55, 'home_appliances_2': 56, 'books_imported': 57, 'drinks': 58, 'cine_photo': 59, 'la_cuisine': 60, 'music': 61, 'home_comfort_2': 62, 'small_appliances_home_oven_and_coffee': 63, 'cds_dvds_musicals': 64, 'dvds_blu_ray': 65, 'flowers': 66, 'arts_and_craftmanship': 67, 'diapers_and_hygiene': 68, 'fashion_childrens_clothes': 69, 'security_and_services': 70, 'Unidentified': 71})

In [38]:
df_join_item_order = df_join_item_order.drop(['product_category_name_english', 'customer_state'], axis=1)

In [39]:
df_join_item_order.count()

order_id                    112650
order_purchase_timestamp    112650
year                        112650
day                         112650
month                       112650
hour                        112650
day_categories              112650
total_order_value           112650
state_number                112650
product_category            112650
dtype: int64

In [60]:
mydata = df_join_item_order.groupby(['year', 'month'])[['total_order_value']].sum().sort_values(by='total_order_value', ascending = True)
mydata = mydata.sort_values(by=['year', 'month'], ascending=True)
mydata['Count of order_id'] = df_join_item_order.groupby(['year', 'month'])[['order_id']].count().sort_values(by=['year', 'month'], ascending = True)

In [61]:
mydata

total_order_value  Count of order_id
year month                                      
2016 9                 562.78                  6
     10              65170.48                363
     12                 19.62                  1
2017 1              162206.06                955
     2              314494.24               1951
     3              479379.70               3000
     4              458803.55               2684
     5              654233.08               4136
     6              551457.86               3583
     7              661289.38               4519
     8              769906.09               4910
     9              868437.75               4831
     10             894949.02               5322
     11            1382776.09               8665
     12             952423.54               6308
2018 1             1257017.33               8208
     2             1147297.84               7672
     3             1315140.71               8217
     4             1328309.64               7975
     5             1332620.65               7925
     6             1161317.09               7078
     7             1204545.35               7092
     8             1115022.12               7248
     9                 166.46                  1

In [ ]:
mydata.to_csv('datamonth.csv')

# Continued to Machine Learning Implementation